<a href="https://colab.research.google.com/github/Munazza-Farees/NITW-SIP2025-Project/blob/main/WS_GTC_FL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas torch sklearn --quiet
!pip install kymatio --quiet

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from kymatio.torch import Scattering1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
URL = "/content/drive/MyDrive/Datasets/crosslayer_ldos_dataset_filtered2.csv"
data = pd.read_csv(URL)

In [4]:
# Select relevant features for wavelet scattering (time-series and statistical)
features = [
    'packet_rate', 'byte_rate', 'inter_arrival_mean', 'inter_arrival_std',
    'flow_duration', 'length_mean', 'length_std', 'packet_size_std',
    'tcp_syn_count', 'tcp_ack_count', 'tcp_flag_entropy', 'rtt_mean', 'rtt_std',
    'payload_entropy', 'source_entropy'
]

X = data[features].values
y = data['label'].values

X = np.nan_to_num(X, nan=np.mean(X, axis=0))

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [22]:
# Set up Wavelet Scattering
J = 2    # Invariance scale
Q = 8     # Quality factor (number of wavelets per octave)
T = X_train.shape[1]      # Length of input signal (number of features)
scattering = Scattering1D(J, T, Q)

X_train_scattering = scattering(X_train)
X_test_scattering = scattering(X_test)

X_train_scattering = X_train_scattering.view(X_train_scattering.size(0), 1, -1)
X_test_scattering = X_test_scattering.view(X_test_scattering.size(0), 1, -1)

/usr/local/lib/python3.11/dist-packages/kymatio/scattering1d/filter_bank.py:218: UserWarning: Signal support is too small to avoid border effects
  warnings.warn('Signal support is too small to avoid border effects')


In [23]:
# 3. Define CNN model
class LDOS_CNN(nn.Module):
    def __init__(self):
        super(LDOS_CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        # Calculate output size after conv and pool
        sequence_length = X_train_scattering.shape[2] - 2  # After conv1 (kernel_size=3)
        sequence_length = sequence_length // 2  # After max pooling (kernel_size=2)
        self.fc_input_size = 32 * sequence_length
        self.fc1 = nn.Linear(self.fc_input_size, 64)
        self.fc2 = nn.Linear(64, 2)  # 2 classes: normal, attack
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
# Explanation:
# - Defines a 1D CNN with one convolutional layer (32 filters), ReLU, and max-pooling.
# - Updates fc_input_size calculation to match reshaped scattering output.
# - Forward pass processes input through conv, pool, and fully connected layers.

# Initialize model, loss function, and optimizer
model = LDOS_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Explanation: Initializes CNN, uses CrossEntropyLoss for binary classification, and Adam optimizer.

# 4. Train the model
num_epochs = 20
batch_size = 32
for epoch in range(num_epochs):
    model.train()
    for i in range(0, X_train_scattering.shape[0], batch_size):
        inputs = X_train_scattering[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
# Explanation: Trains model for 20 epochs, processing batches of size 32, computing loss, and updating weights.

# 5. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_scattering)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).float().mean()
    print(f'Test Accuracy: {accuracy.item()*100:.2f}%')
# Explanation: Evaluates model on test data and computes accuracy.

# 6. Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Models/cnn_ldos_model.pth')
# Explanation: Saves trained model weights to a file.

Epoch 1, Loss: 0.5417799949645996
Epoch 2, Loss: 0.3870615065097809
Epoch 3, Loss: 0.23589277267456055
Epoch 4, Loss: 0.10661576688289642
Epoch 5, Loss: 0.054958608001470566
Epoch 6, Loss: 0.0417223758995533
Epoch 7, Loss: 0.03819148242473602
Epoch 8, Loss: 0.036348067224025726
Epoch 9, Loss: 0.035300083458423615
Epoch 10, Loss: 0.03541140258312225
Epoch 11, Loss: 0.03405031934380531
Epoch 12, Loss: 0.03348416090011597
Epoch 13, Loss: 0.034156907349824905
Epoch 14, Loss: 0.03457849472761154
Epoch 15, Loss: 0.034485042095184326
Epoch 16, Loss: 0.03493461385369301
Epoch 17, Loss: 0.035082992166280746
Epoch 18, Loss: 0.03500992804765701
Epoch 19, Loss: 0.03506786748766899
Epoch 20, Loss: 0.03506803140044212
Test Accuracy: 96.36%
